In [90]:
# Package usado para cargar datos desde un archivo h5
from h5py import File

# Package para trabajar con arreglos
import numpy as np

from numpy import loadtxt, mean, array, concatenate, load, zeros, vstack, ravel
import matplotlib.pyplot as plt
# bosignalsnotebooks python package
import biosignalsnotebooks as bsnb
from numpy import max, min, average, std, sum, sqrt, where, argmax, absolute, random 
from numpy import loadtxt, logical_and, trapz
from scipy.integrate import cumtrapz
from scipy.signal import welch
from scipy.stats import kurtosis, skew
from bokeh.plotting import figure, show
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, accuracy_score

# **Tratamiento de señal de EEG**

In [10]:
#EEG_ojos_cerrados
File_Name = "EEG1.h5"
h5_object = File(File_Name)

In [31]:
# Cargar Datos
data, header = bsnb.load("EEG1.h5", get_header=True)

In [13]:
print(data)
print(header)

{'CH4': array([656, 647, 643, ..., 543, 547, 557], dtype=uint32)}
{'channels': array([4], dtype=int32), 'comments': '', 'date': '2023-4-19', 'device': 'bitalino_rev', 'device connection': 'BTH98:D3:61:FD:78:31', 'device name': '98:D3:61:FD:78:31', 'digital IO': array([0, 0, 1, 1], dtype=int32), 'firmware version': 1282, 'resolution': array([ 4,  1,  1,  1,  1, 10], dtype=int32), 'sampling rate': 1000, 'sync interval': 2, 'time': '11:36:12.646', 'sensor': ['EEGBITREV'], 'column labels': {4: 'channel_4'}}


In [15]:
signal = data["CH4"]
time = bsnb.generate_time(signal, header["sampling rate"])

In [16]:
# Nueva Bokeh figure
bokeh_figure = figure(x_axis_label = 'Time (s)', y_axis_label='Raw Data')

In [17]:
bokeh_figure.line(time, signal, legend_label="Datos Originales")

GlyphRenderer(id='1037', ...)

In [18]:
show(bokeh_figure)

In [20]:
# **Filtrado de señal**
#Ventana de tiempo
sampling_rate=1000
t_start= 0
sample_start=t_start*sampling_rate

t_end=30
sample_end=t_end*sampling_rate

In [21]:
#Frecuencia de corte
low_cutoff_wide=3 #Lower cutoff frequency for bandpass filter (Hz)
high_cuttoff_wide=30 #Upper cutoff frequency for bandpass filter (Hz)


In [48]:
filtered_signal=bsnb.bandpass(signal,low_cutoff_wide,high_cuttoff_wide, order=2, fs=1000)

# **Detección de ERP**


In [32]:
File_Name2 = "EEG2.h5"
h5_object2 = File(File_Name)

In [39]:
data2, header2 = bsnb.load("EEG2.h5", get_header=True)
ESTIMULO = data["CH4"]

In [41]:
#Estímulo
signal_ac= ESTIMULO - mean(ESTIMULO)

#Generando los ejes de tiempo:
time_eeg= bsnb.generate_time(signal)
time_a= bsnb.generate_time(ESTIMULO)

time_a_down= time_a[::10]
signal_ac_down=signal_ac[::10]

#Estimulo sonoro:
bsnb.plot([time_a_down],[signal_ac_down], y_axis_label="Value RAW", legend="Acoustic Simuli RAW")

# **Extracción de la banda Alfa**

In [50]:
sr= 1000
win= 4*sr

In [51]:
#Define Frequency Band limits:
freq_low = 8 #lower limit for alpha band
freq_high = 12 #Upper limit for alpha band 

In [52]:
filtered_signal_estimulo=bsnb.bandpass(ESTIMULO,low_cutoff_wide,high_cuttoff_wide, order=2, fs=1000)

In [53]:
freq_axis_reposo, power_spect_reposo = welch(filtered_signal, sr, nperseg=win)

In [54]:
freq_axis_estimulo, power_spect_estimulo = welch(filtered_signal_estimulo, sr, nperseg=win)

In [58]:
#Find the intersection Values of the alpha band in the frequency vector [Eyes Closed]
idx_alpha_reposo= logical_and(freq_axis_reposo >= freq_low, freq_axis_reposo <= freq_high)

#Find the intersection Values of the alpha band in the frequency vector [Eyes Opened]
idx_alpha_estimulo = logical_and(freq_axis_estimulo >= freq_low, freq_axis_estimulo <= freq_high)

In [59]:
freq_res_reposo = freq_axis_reposo[1] - freq_axis_reposo[0] 

#Compute the Absolute Power with numpy.trapz:
alpha_power_reposo = trapz(power_spect_reposo[idx_alpha_reposo],dx=freq_res_reposo)

In [60]:
freq_res_estimulo = freq_axis_estimulo[1] - freq_axis_estimulo[0] 

#Compute the Absolute Power with numpy.trapz:
alpha_power_estimulo = trapz(power_spect_estimulo[idx_alpha_estimulo],dx=freq_res_estimulo)

In [61]:
print(alpha_power_reposo,alpha_power_estimulo)

761.7919274149122 761.7919274149122


# **Diferencia entre señales**

In [74]:

# Load the ECG signal
ECG = bsnb.load("ecg_20_sec_1000_Hz.h5")['CH1']
# Normalise ECG signal
ECG = ECG - mean(ECG); ECG /= std(ECG)
ECG_time = bsnb.generate_time(ECG)
bsnb.plot(ECG_time, ECG)

In [75]:
# Load the EMG signal
EMG = bsnb.load("emg_bursts.h5")['CH3']
# Normalise EMG signal
EMG = EMG - mean(EMG); EMG /= std(EMG)
EMG_time = bsnb.generate_time(EMG)
bsnb.plot(EMG_time, EMG)

In [76]:
# Load other signal
EEG = bsnb.load("signal_sample_single_hub_EEG_2018_7_4.h5")['CH1']
# Normalise other signal
EEG = EEG - mean(EEG); EEG /= std(EEG)
EEG_time = bsnb.generate_time(EEG)
bsnb.plot(EEG_time, EEG)

In [77]:
# Load another signal
ACO = bsnb.load("sync_cable_acoustic.h5")
# Get the mac address of one of the devices used in this acquisition
mac_address = list(ACO.keys())[0]
# Load the the signal corresponding to the first channel of the device
ACO = ACO[mac_address]['CH1']
# Normalise another signal
ACO = ACO - mean(ACO); ACO /= std(ACO)
ACO_time = bsnb.generate_time(ACO)
bsnb.plot(ACO_time, ACO)

In [94]:
sampling_rate = 1000 # Hz
time_window = 1 # seconds
overlap = 0.9

ECG_windows = bsnb.windowing(ECG, sampling_rate, time_window, overlap)
EMG_windows = bsnb.windowing(EMG, sampling_rate, time_window, overlap)
EEG_windows = bsnb.windowing(EEG, sampling_rate, time_window, overlap)
ACO_windows = bsnb.windowing(ACO, sampling_rate, time_window, overlap)
signal_windows = bsnb.windowing(filtered_signal, sampling_rate, time_window, overlap)

# Get the number of time windows of the shortest signal in order to concatenate all signals to the same number of time windows.
index = min([ECG_windows.shape[0], EMG_windows.shape[0], EEG_windows.shape[0], ACO_windows.shape[0],signal_windows.shape[0]])

# Concatenate all signals to the same number of time windows.
ECG_windows = ECG_windows[:index]
EMG_windows = EMG_windows[:index]
EEG_windows = EEG_windows[:index]
ACO_windows = ACO_windows[:index]
signal_windows = signal_windows[:index]

In [98]:
# Defining the functions applied to each time window in order to extract features
func = [mean, std, kurtosis, skew, bsnb.zero_crossing_rate]

ECG_features = bsnb.features_extraction(ECG_windows, func)
EMG_features = bsnb.features_extraction(EMG_windows, func)
EEG_features = bsnb.features_extraction(EEG_windows, func)
ACO_features = bsnb.features_extraction(ACO_windows, func)
signal_features = bsnb.features_extraction(signal_windows, func)


# Get the number of samples of each type of signal - in this case, our classes.
lenght_ECG = ECG_features.shape[0]
lenght_EMG = EMG_features.shape[0]
lenght_EEG = EEG_features.shape[0]
lenght_ACO = ACO_features.shape[0]
lenght_signal = signal_features.shape[0]

# Build the samples array containing all samples from each signal.
samples = vstack([ECG_features, EMG_features, EEG_features, ACO_features])
samples2 = vstack([ECG_features, EMG_features, signal_features, ACO_features])

# Build the classes array to assign each sample to its class. 
classes = concatenate([lenght_ECG*['ECG'], lenght_EMG*['EMG'], lenght_EEG*['Other'], lenght_ACO*['Other']])
classes2 = concatenate([lenght_ECG*['ECG'], lenght_EMG*['EMG'], lenght_signal*['Other'], lenght_ACO*['Other']])

In [85]:
model = ShuffleSplit(n_splits=10, train_size=.9, test_size=.1)

In [100]:
acc = []
for train_index, test_index in model.split(samples):
    
    # For each iteration, we divide our dataset in train and test set.
    samples_train, samples_test = samples[train_index], samples[test_index]
    labels_train, labels_test = classes[train_index], classes[test_index]
    
    # Build the random forest clasdsifier.
    random_forest = RandomForestClassifier(n_estimators=1000, criterion='gini')
    
    # Train the classifier on the training set.
    random_forest = random_forest.fit(samples_train, ravel(labels_train))
    
    # Test the classifier on the testing set.
    results = random_forest.predict(samples_test)

    # This step is not necessary for the classification procedure, but is important to store the values 
    # of accuracy to calculate the mean and standard deviation values and evaluate the performance of the classifier.
    acc.append(accuracy_score(labels_test, results)*100)

ValueError: ignored

In [102]:
acc2 = []
for train_index, test_index in model.split(samples2):
    
    # For each iteration, we divide our dataset in train and test set.
    samples_train, samples_test = samples2[train_index], samples2[test_index]
    labels_train, labels_test = classes2[train_index], classes2[test_index]
    
    # Build the random forest clasdsifier.
    random_forest = RandomForestClassifier(n_estimators=1000, criterion='gini')
    
    # Train the classifier on the training set.
    random_forest = random_forest.fit(samples_train, ravel(labels_train))
    
    # Test the classifier on the testing set.
    results = random_forest.predict(samples_test)

    # This step is not necessary for the classification procedure, but is important to store the values 
    # of accuracy to calculate the mean and standard deviation values and evaluate the performance of the classifier.
    acc.append(accuracy_score(labels_test, results)*100)

ValueError: ignored

In [104]:
print("Accuracy: ", mean(acc), "+-", std(acc), "%")

Accuracy:  nan +- nan %


In [105]:
print("Accuracy: ", mean(acc2), "+-", std(acc2), "%")

Accuracy:  nan +- nan %
